In [1]:
import tensorflow as tf
# from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import backend as K

from src import data
from src import error_function

jobs = 6 # number of cores
config = tf.ConfigProto(intra_op_parallelism_threads=jobs,
                     inter_op_parallelism_threads=jobs,
                     allow_soft_placement=True,
                     device_count={'CPU': jobs})
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [2]:
## loading the model
model = keras.models.load_model('CRNN.h5',         
        custom_objects={ 
            'class_mae': error_function.class_mae,
            'exp': keras.backend.exp
        })

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/Users/jordynaus/Documents/Artificial Intelligence/Master/2021 Sem. 2/Automatic Speech Recognition/asr/lib/python3.7/site-packages/keras/engine/base_layer.py:1179: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="conv1", activity_regularizer=None, trainable=True, activation="relu", kernel_size=(3, 3), filters=64, strides=[1, 1], padding="valid", data_format="channels_first", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None, use_bias=True)`
  return cls(**config)
/Users/jordynaus/Documents/Artificial Intelligence/Master/2021 Sem. 2/Automatic Speech Recognition/asr/lib/python3.7/site-packages/keras/engine/base_layer.py:1179: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="conv2", activity_regularizer=None, trainable=True, activation="relu", kernel_size=(3, 3), filters=32, strides=[1, 1], padding="valid", data_format="channels_first", kernel_initializer="glorot

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/Users/jordynaus/Documents/Artificial Intelligence/Master/2021 Sem. 2/Automatic Speech Recognition/asr/lib/python3.7/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero1 (ZeroPadding2D)        (None, 1, 500, 201)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 498, 199)      640       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 496, 197)      18464     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 32, 165, 65)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 128, 163, 63)      36992     
_________________________________________________________________
conv4 (Conv2D)               (None, 64, 161, 61)       73792     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 64, 53, 20)       

In [4]:
## load dataset
train_dataset, val_dataset = data.create_tensorflow_dataset('./data/trainset100/stft/', batch_size=32, val_split = 0.3)

Found 7860 images belonging to 11 classes.
Found 3363 images belonging to 11 classes.


In [5]:
dataset[0][1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

In [6]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs = 50)

Epoch 1/50
246/246 [==============================] - 1947s 8s/step - loss: 2.4246 - accuracy: 0.1992 - class_mae: 2.3374 - val_loss: 2.1809 - val_accuracy: 0.1389 - val_class_mae: 2.0660
Epoch 2/50
246/246 [==============================] - 2046s 8s/step - loss: 1.7563 - accuracy: 0.3036 - class_mae: 1.8333 - val_loss: 1.4305 - val_accuracy: 0.3259 - val_class_mae: 1.8491
Epoch 3/50
  7/246 [..............................] - ETA: 29:44 - loss: 1.4764 - accuracy: 0.4107 - class_mae: 1.2857

KeyboardInterrupt: 

In [ ]:
model.save('CRNN_100.h5', overwrite=True, include_optimizer=True)